## This notebook will be mainly used for the capstone project

In [4]:
import pandas as pd
import numpy as np

In [5]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Exploration and clustering of the neighborhoods in Toronto.

#### Fistly, we will import everything we might need

In [ ]:
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

#### Then we should fetch data from <a link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'> Wikipedia</a> with PostalCode, Borough, and Neighborhood

In [103]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
lst = pd.read_html(url)

In [111]:
df=pd.DataFrame(lst[0])

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [112]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [113]:
df = df.drop(df[df['Borough']=='Not assigned'].index)

In [ ]:
df.head()

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [114]:
df = df.groupby('Postal Code').agg({'Borough': list,'Neighborhood': list})
df = df.reset_index()
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: ','.join(x))
df['Borough'] = df['Borough'].apply(lambda x: ','.join(x))                                     

In [117]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [116]:
df.shape

(103, 3)